In [3]:
from ber_metrics import BEREstimator
import numpy as np
import pickle
import pandas as pd


## Adult Income

In [5]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/adult_income/processed_data_nonsingular.pkl', 'rb'))
X_train = X_train.drop(labels=['race_Asian-Pac-Islander', 'race_Black', 'race_Other', 'race_White', 'sex_Male'], axis=1)
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
adult_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(adult_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(adult_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(adult_estimator.nn_bound()))


(30162, 91)
Bhattacharrya upper bound: 6.426276272985858e-44
Mahalanobis upper bound: 0.23657243359215477


In [10]:
# for the different groups
print("----Decoupled Race-----")
X_train, y_train, X_test, y_test = pickle.load(open('Data/adult_income/processed_data_nonsingular.pkl', 'rb'))
X_train, y_train = X_train[X_train['race_Black']==1], y_train[X_train['race_Black']==1]
X_test, y_test = X_test[X_test['race_Black']==1], y_test[X_test['race_Black']==1]
X_train = X_train.drop(labels=['race_Asian-Pac-Islander', 'race_Black', 'race_Other', 'race_White', 'sex_Male'], axis=1)
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
adult_estimator = BEREstimator(X_train, y_train)
print("Black")
print("Mahalanobis upper bound: {}".format(adult_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(adult_estimator.nn_bound()))

X_train, y_train, X_test, y_test = pickle.load(open('Data/adult_income/processed_data_nonsingular.pkl', 'rb'))
X_train, y_train = X_train[X_train['race_White']==1], y_train[X_train['race_White']==1]
X_test, y_test = X_test[X_test['race_White']==1], y_test[X_test['race_White']==1]
X_train = X_train.drop(labels=['race_Asian-Pac-Islander', 'race_Black', 'race_Other', 'race_White', 'sex_Male'], axis=1)
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
adult_estimator = BEREstimator(X_train, y_train)
print("White")
print("Mahalanobis upper bound: {}".format(adult_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(adult_estimator.nn_bound()))

print("-----Decoupled Sex-------")
X_train, y_train, X_test, y_test = pickle.load(open('Data/adult_income/processed_data_nonsingular.pkl', 'rb'))
X_train, y_train = X_train[X_train['sex_Male']==1], y_train[X_train['sex_Male']==1]
X_test, y_test = X_test[X_test['sex_Male']==1], y_test[X_test['sex_Male']==1]
X_train = X_train.drop(labels=['race_Asian-Pac-Islander', 'race_Black', 'race_Other', 'race_White', 'sex_Male'], axis=1)
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
adult_estimator = BEREstimator(X_train, y_train)
print("Male")
print("Mahalanobis upper bound: {}".format(adult_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(adult_estimator.nn_bound()))

X_train, y_train, X_test, y_test = pickle.load(open('Data/adult_income/processed_data_nonsingular.pkl', 'rb'))
X_train, y_train = X_train[X_train['sex_Male']==0], y_train[X_train['sex_Male']==0]
X_test, y_test = X_test[X_test['sex_Male']==0], y_test[X_test['sex_Male']==0]
X_train = X_train.drop(labels=['race_Asian-Pac-Islander', 'race_Black', 'race_Other', 'race_White', 'sex_Male'], axis=1)
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
adult_estimator = BEREstimator(X_train, y_train)
print("Male")
print("Mahalanobis upper bound: {}".format(adult_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(adult_estimator.nn_bound()))


-----Decoupled Sex-------
Male


/Users/AngelinaWang/Documents/School/G2/COS597E/final_project/cos597efinalproject/ber_metrics.py:30: RuntimeWarning: invalid value encountered in true_divide
  self.x = (self.x - mu) / std # standardize feature scale


LinAlgError: SVD did not converge

## COMPAS Recidivism

In [3]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/compas/processed_violent_nonsingular.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
violent_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(violent_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(violent_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(violent_estimator.nn_bound()))

Bhattacharrya upper bound: 0.060966314167847147
Mahalanobis upper bound: 0.19586095536150863


In [4]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/compas/processed_violent.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
violent_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(violent_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(violent_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(violent_estimator.nn_bound()))

Bhattacharrya upper bound: 0.02596945557312589
Mahalanobis upper bound: 0.18821591060670279
Nearest Neighbor upper bound: 0.16203143893591293


In [5]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/compas/processed_arrest_nonsingular.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
violent_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(violent_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(violent_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(violent_estimator.nn_bound()))

Bhattacharrya upper bound: 2.5016397436853592e-05
Mahalanobis upper bound: 0.4223692451173532


In [5]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/compas/processed_arrest.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
violent_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(violent_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(violent_estimator.mahalanobis_bound()))
print("Nearest Neighbor upper bound: {}".format(violent_estimator.nn_bound()))

Bhattacharrya upper bound: 1.3432602268871714e-08
Mahalanobis upper bound: 0.4158120026746693
Nearest Neighbor upper bound: 0.441596130592503
